In [2]:
import pandas as pd
import numpy as np
from scipy import linalg as LA
from scipy import stats as ST
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from statsmodels.distributions.empirical_distribution import ECDF
from myfunc import OLS_mean, kernel_test, loc_poly,my_bootstrap

In [3]:
def OLS_parameter(X, Y):
    beta = sm.OLS(Y, sm.add_constant(X)).fit().params.T
    return beta

 # Import Data
 1. Return data. Here I use S&P500 constituents, can be extended to more stocks
 2. Import Factor data from Factor Zoo

In [4]:
DATA = pd.read_csv(
    'HPRET_monthly_correspondint_to_factor_zoo_all.csv')
DATA.date = pd.to_datetime(DATA.date, format='%Y%m%d')

DATA = DATA.drop(DATA[DATA.RET == 'C'].index)
DATA = DATA.drop(DATA[DATA.RET == 'B'].index)
DATA.RET = DATA['RET'].astype('float')
RET = DATA.pivot_table('RET', index='PERMNO', columns='date')
RET = RET.dropna(0).transpose()
RET.iloc[0:5,0:5]

PERMNO,10065,10145,10516,10656,10866
date,,,,,
1976-07-30,0.000000,0.003215,-0.108108,-0.076190,0.032967
1976-08-31,0.000000,-0.046154,-0.134343,0.185567,-0.101064
1976-09-30,-0.002151,0.051020,-0.070175,-0.017391,-0.008284
1976-10-29,0.076087,-0.048544,0.044025,-0.053097,-0.030303
1976-11-30,-0.030303,-0.052381,-0.003614,0.074766,0.087500


In [5]:
FACTOR = pd.read_csv(
    'factors_zoo.csv')
FACTOR.rename(columns={'  Date': 'date'}, inplace=True)
FACTOR.date = pd.to_datetime(FACTOR.date, format='%Y%m%d')
FACTOR = FACTOR.pivot_table(index = 'date')
FACTOR = FACTOR.dropna(axis=1)
FACTOR.iloc[0:5,0:5]

,BAB,CMA,HML,HML_Devil,HXZ_IA
date,,,,,
1976-07-30,0.018660,0.0029,0.0174,0.022995,0.009213
1976-08-31,0.013834,-0.0059,0.0079,0.013759,-0.005963
1976-09-30,0.017247,-0.0114,-0.0029,0.000902,-0.014150
1976-10-29,0.011174,-0.0031,-0.0013,0.002927,-0.011151
1976-11-30,0.004498,0.0008,0.0151,0.013638,-0.002691


 # Individual test of nonlinearity

In [ ]:
Result = np.zeros(147)
Critical_left = np.zeros(147)
Critical_right = np.zeros(147)
average_ret = np.array(RET.mean())
for i in range(0,1):
    beta = np.array(OLS_parameter(FACTOR.iloc[:,i], RET).iloc[:, 1])
    # average_ret = np.array(RET.iloc[8,:])
    # plt.figure
    # plt.scatter(beta, average_ret)
    Result[i], Critical_left[i], Critical_right[i] = my_bootstrap(beta, average_ret)
    print(i)
# pd.DataFrame([Result, Critical_left,Critical_right]).to_csv('result.csv')


In [8]:
result = pd.read_csv('result.csv')
(result.iloc[0, :] < result.iloc[1,:]).sum()
(result.iloc[0, :] > result.iloc[2, :]).sum()

0